In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

!. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2. How many Stray Animal cases are there?

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

4. Convert the council_district column to a string column.

5. Extract the year from the case_closed_date column.

6. Convert num_days_late from days to hours in new columns num_hours_late.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?

In [2]:
spark = SparkSession.builder.getOrCreate()

In [5]:
from pyspark.sql.types import StructType, StructField, StringType

In [18]:
schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
        
    ]
)

In [19]:
source = spark.read.csv("source.csv",
                        header =True,
                        schema = schema)
         

In [20]:
case_csv = spark.read.csv("case.csv", header = True, inferSchema = True)

In [21]:
dept_csv = spark.read.csv("dept.csv",
                     header = True,
                     schema = schema)

In [29]:
source.write.json("source_json", mode = "overwrite")

In [30]:
case.write.json("case_json", mode = "overwrite")

In [31]:
dept.write.json("dept_json", mode = "overwrite")

In [32]:
source.write.csv("source_csv", mode = "overwrite")

case.write.csv("case_csv", mode = "overwrite")

dept.write.csv("dept_csv", mode = "overwrite")

# 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [35]:
case.select("SLA_due_date").show(5)

+------------+
|SLA_due_date|
+------------+
|9/26/20 0:42|
| 1/5/18 8:30|
| 1/5/18 8:30|
|1/17/18 8:30|
| 1/1/18 4:34|
+------------+
only showing top 5 rows



In [36]:
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [37]:
case.groupBy("case_closed", "case_late").count().show()

+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [41]:
fmt = "M/d/yy H:mm"

case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))\
.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))\
.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [42]:
# getting the number of days late
case = case.withColumn("num_weeks_late", expr("num_days_late / 7"))

In [44]:
case.select(case.columns[1:4]).show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [45]:
case.select("num_days_late", "num_weeks_late").show(3)

+-------------------+--------------------+
|      num_days_late|      num_weeks_late|
+-------------------+--------------------+
| -998.5087616000001|        -142.6441088|
|-2.0126041669999997|-0.28751488099999994|
|       -3.022337963|-0.43176256614285713|
+-------------------+--------------------+
only showing top 3 rows



In [49]:
case.groupBy("case_due_date").agg(min("case_due_date")).withColumn().show(5)

+-------------------+-------------------+
|      case_due_date| min(case_due_date)|
+-------------------+-------------------+
|2018-01-02 03:25:00|2018-01-02 03:25:00|
|2018-01-06 16:41:00|2018-01-06 16:41:00|
|2018-01-11 14:45:00|2018-01-11 14:45:00|
|2020-09-27 17:39:00|2020-09-27 17:39:00|
|2018-01-18 08:49:00|2018-01-18 08:49:00|
+-------------------+-------------------+
only showing top 5 rows



In [51]:
# gettting age as a feature
case = case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))

In [52]:
case.select("case_age", "case_opened_date").show(5)

+--------+-------------------+
|case_age|   case_opened_date|
+--------+-------------------+
|    1505|2018-01-01 00:42:00|
|    1505|2018-01-01 00:46:00|
|    1505|2018-01-01 00:48:00|
|    1505|2018-01-01 01:29:00|
|    1505|2018-01-01 01:34:00|
+--------+-------------------+
only showing top 5 rows



# 2. How many Stray Animal cases are there?

In [53]:
dept.show(5)

+--------------------+--------------------+
|           source_id|     source_username|
+--------------------+--------------------+
|     311 Call Center|    Customer Service|
|               Brush|Solid Waste Manag...|
|     Clean and Green|Parks and Recreation|
|Clean and Green N...|Parks and Recreation|
|    Code Enforcement|Code Enforcement ...|
+--------------------+--------------------+
only showing top 5 rows



In [54]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [55]:
case.show(5)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+--------------------+--------+
|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|      num_weeks_late|case_age|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+--------------------+--------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|        -14